# MULTISITE MACHINE LEARNING ANALYSIS - DAMIAN KOTEVSKI

In [ ]:
# library requirements
import pandas as pd, numpy as np
from numpy import isnan
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, brier_score_loss
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display
import pickle
import lifelines
from lifelines.utils import concordance_index
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test, multivariate_logrank_test
from lifelines.plotting import add_at_risk_counts

In [ ]:
# load data
hnc_df = pd.read_csv("/PATH/TO/FILE.csv")
hpc_df = pd.read_csv("/PATH/TO/FILE.csv")
lyx_df = pd.read_csv("/PATH/TO/FILE.csv")
npc_df = pd.read_csv("/PATH/TO/FILE.csv")
occ_df = pd.read_csv("/PATH/TO/FILE.csv")
opc_df = pd.read_csv("/PATH/TO/FILE.csv")

In [ ]:
# define imputer
imputer = IterativeImputer(estimator=BayesianRidge(),
                           n_nearest_features=None, 
                           imputation_order='ascending', # uses features with fewest missing values first
                           initial_strategy='most_frequent') # used for categorical data

In [ ]:
# fit the imputer on the dataset
imputer.fit(hnc_df)
imputer.fit(hpc_df)
imputer.fit(lyx_df)
imputer.fit(npc_df)
imputer.fit(occ_df)
imputer.fit(opc_df)

In [ ]:
# transform the dataset
transformed_hnc_df = imputer.transform(hnc_df)
transformed_hpc_df = imputer.transform(hpc_df)
transformed_lyx_df = imputer.transform(lyx_df)
transformed_npc_df = imputer.transform(npc_df)
transformed_occ_df = imputer.transform(occ_df)
transformed_opc_df = imputer.transform(opc_df)

# print total missing
# print('Missing: %d' % sum(isnan(transformed_hnc_df).flatten()))
# print('Missing: %d' % sum(isnan(transformed_hpc_df).flatten()))
# print('Missing: %d' % sum(isnan(transformed_lyx_df).flatten()))
# print('Missing: %d' % sum(isnan(transformed_npc_df).flatten()))
# print('Missing: %d' % sum(isnan(transformed_occ_df).flatten()))
# print('Missing: %d' % sum(isnan(transformed_opc_df).flatten()))

In [ ]:
# create complete dataframe with imputed data
complete_hnc_df = pd.DataFrame(imputer.transform(hnc_df))
complete_hpc_df = pd.DataFrame(imputer.transform(hpc_df))
complete_lyx_df = pd.DataFrame(imputer.transform(lyx_df))
complete_npc_df = pd.DataFrame(imputer.transform(npc_df))
complete_occ_df = pd.DataFrame(imputer.transform(occ_df))
complete_opc_df = pd.DataFrame(imputer.transform(opc_df))

In [ ]:
# define columns for dataframe with imputed data
complete_hnc_df.columns = ['PatientID', 'Hospital', 'Age', 'Gender', 'TumourSite', 'Tclassification',
                       'Nclassification', 'EQD2T', 'SurvivalMonths', 'TwoYearHNCDeath']
complete_hpc_df.columns = ['PatientID', 'Hospital', 'Age', 'Gender', 'TumourSite', 'Tclassification',
                       'Nclassification', 'EQD2T', 'SurvivalMonths', 'TwoYearHNCDeath']
complete_lyx_df.columns = ['PatientID', 'Hospital', 'Age', 'Gender', 'TumourSite', 'Tclassification',
                       'Nclassification', 'EQD2T', 'SurvivalMonths', 'TwoYearHNCDeath']
complete_npc_df.columns = ['PatientID', 'Hospital', 'Age', 'Gender', 'TumourSite', 'Tclassification',
                       'Nclassification', 'EQD2T', 'SurvivalMonths', 'TwoYearHNCDeath']
complete_occ_df.columns = ['PatientID', 'Hospital', 'Age', 'Gender', 'TumourSite', 'Tclassification',
                       'Nclassification', 'EQD2T', 'SurvivalMonths', 'TwoYearHNCDeath']
complete_opc_df.columns = ['PatientID', 'Hospital', 'Age', 'Gender', 'TumourSite', 'Tclassification',
                       'Nclassification', 'EQD2T', 'SurvivalMonths', 'TwoYearHNCDeath']

# print dataframe with imputed data
# print(complete_hnc_df)
# print(complete_hpc_df)
# print(complete_lyx_df)
# print(complete_npc_df)
# print(complete_occ_df)
# print(complete_opc_df)

In [ ]:
# round data in imputed dataframe
completed_hnc_df = complete_hnc_df.abs().round()
completed_hpc_df = complete_hpc_df.abs().round()
completed_lyx_df = complete_lyx_df.abs().round()
completed_npc_df = complete_npc_df.abs().round()
completed_occ_df = complete_occ_df.abs().round()
completed_opc_df = complete_opc_df.abs().round()

# print rounded dataframe
# print(completed_hnc_df)
# print(completed_hpc_df)
# print(completed_lyx_df)
# print(completed_npc_df)
# print(completed_occ_df)
# print(completed_opc_df)

In [ ]:
# define the modeling pipeline to evaluate imputation
model = RandomForestClassifier()
imputer = IterativeImputer()
pipeline = Pipeline(steps=[('i', imputer), ('m', model)])

# split into input and output elements
hnc_data = hnc_df.values
ix = [i for i in range(hnc_data.shape[1])]
X_hnc, y_hnc = hnc_data[:, ix], hnc_data[:, 4] # based on 'Age' column

hpc_data = hpc_df.values
ix = [i for i in range(hpc_data.shape[1])]
X_hpc, y_hpc = hpc_data[:, ix], hpc_data[:, 4] # based on 'Age' column

lyx_data = lyx_df.values
ix = [i for i in range(lyx_data.shape[1])]
X_lyx, y_lyx = lyx_data[:, ix], lyx_data[:, 4] # based on 'Age' column

npc_data = npc_df.values
ix = [i for i in range(npc_data.shape[1])]
X_npc, y_npc = npc_data[:, ix], npc_data[:, 4] # based on 'Age' column

occ_data = occ_df.values
ix = [i for i in range(occ_data.shape[1])]
X_occ, y_occ = occ_data[:, ix], occ_data[:, 4] # based on 'Age' column

opc_data = opc_df.values
ix = [i for i in range(opc_data.shape[1])]
X_opc, y_opc = opc_data[:, ix], opc_data[:, 4] # based on 'Age' column

# define model evaluation
cv = RepeatedStratifiedKFold(n_splits=2, n_repeats=2, random_state=1)

# evaluate model
hnc_scores = cross_val_score(pipeline, X_hnc, y_hnc, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)'% (hnc_scores.mean(), hnc_scores.std()))

hpc_scores = cross_val_score(pipeline, X_hpc, y_hpc, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)'% (hpc_scores.mean(), hpc_scores.std()))

lyx_scores = cross_val_score(pipeline, X_lyx, y_lyx, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)'% (lyx_scores.mean(), lyx_scores.std()))

npc_scores = cross_val_score(pipeline, X_npc, y_npc, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)'% (npc_scores.mean(), npc_scores.std()))

occ_scores = cross_val_score(pipeline, X_occ, y_occ, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)'% (occ_scores.mean(), occ_scores.std()))

opc_scores = cross_val_score(pipeline, X_opc, y_opc, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# print('Mean Accuracy: %.3f (%.3f)'% (opc_scores.mean(), opc_scores.std()))

# test imputation
# TwoYearHNCDeath_occ_unrounded = complete_occ_df['TwoYearHNCDeath'].value_counts()
# print("Frequency of TwoYearHNCDeath")
# print(TwoYearHNCDeath_occ_unrounded)
# TwoYearHNCDeath_occ_rounded = completed_hnc_df['TwoYearHNCDeath'].value_counts()
# print("Frequency of TwoYearHNCDeath")
# print(TwoYearHNCDeath_occ_rounded)

# oversampling

In [ ]:
# split the dataset into input and output features
X_hnc = completed_hnc_df.drop(axis=1, columns=['PatientID', 'SurvivalMonths', 'TwoYearHNCDeath'])
y_hnc = completed_hnc_df['TwoYearHNCDeath']

X_hpc = completed_hpc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'SurvivalMonths', 'TwoYearHNCDeath'])
y_hpc = completed_hpc_df['TwoYearHNCDeath']

X_lyx = completed_lyx_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'SurvivalMonths', 'TwoYearHNCDeath'])
y_lyx = completed_lyx_df['TwoYearHNCDeath']

X_npc = completed_npc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'SurvivalMonths', 'TwoYearHNCDeath'])
y_npc = completed_npc_df['TwoYearHNCDeath']

X_occ = completed_occ_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'SurvivalMonths', 'TwoYearHNCDeath'])
y_occ = completed_occ_df['TwoYearHNCDeath']

X_opc = completed_opc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'SurvivalMonths', 'TwoYearHNCDeath'])
y_opc = completed_opc_df['TwoYearHNCDeath']

In [ ]:
# create balanced dataset
smote = SMOTE(random_state=1, k_neighbors=3) 
X_hnc_smote, y_hnc_smote = smote.fit_resample(X_hnc, y_hnc)
X_hpc_smote, y_hpc_smote = smote.fit_resample(X_hpc, y_hpc)
X_lyx_smote, y_lyx_smote = smote.fit_resample(X_lyx, y_lyx)
X_npc_smote, y_npc_smote = smote.fit_resample(X_npc, y_npc)
X_occ_smote, y_occ_smote = smote.fit_resample(X_occ, y_occ)
X_opc_smote, y_opc_smote = smote.fit_resample(X_opc, y_opc)

In [ ]:
# check dataset is balanced
# print(f'''Shape of X before SMOTE: {X_hnc.shape}
# Shape of X after SMOTE: {X_hnc_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y_hnc.shape}
# Shape of y after SMOTE: {y_hnc_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
# y_hnc_smote.value_counts(normalize=True) * 100

# print(f'''Shape of X before SMOTE: {X_hpc.shape}
# Shape of X after SMOTE: {X_hpc_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y_hpc.shape}
# Shape of y after SMOTE: {y_hpc_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
# y_hpc_smote.value_counts(normalize=True) * 100

# print(f'''Shape of X before SMOTE: {X_lyx.shape}
# Shape of X after SMOTE: {X_lyx_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y_lyx.shape}
# Shape of y after SMOTE: {y_lyx_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
# y_lyx_smote.value_counts(normalize=True) * 100

# print(f'''Shape of X before SMOTE: {X_npc.shape}
# Shape of X after SMOTE: {X_npc_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y_npc.shape}
# Shape of y after SMOTE: {y_npc_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
# y_npc_smote.value_counts(normalize=True) * 100

# print(f'''Shape of X before SMOTE: {X_occ.shape}
# Shape of X after SMOTE: {X_occ_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y_occ.shape}
# Shape of y after SMOTE: {y_occ_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
# y_occ_smote.value_counts(normalize=True) * 100

# print(f'''Shape of X before SMOTE: {X_opc.shape}
# Shape of X after SMOTE: {X_opc_smote.shape}''')
# print(f'''Shape of y before SMOTE: {y_opc.shape}
# Shape of y after SMOTE: {y_opc_smote.shape}''')
# print('\nBalance of positive and negative classes (%):')
# y_opc_smote.value_counts(normalize=True) * 100

# prediction - hnc dataset

In [ ]:
# initiate random forest model
rf = RandomForestClassifier(random_state=0)
# split into 80% train and 20% test using oversampled dataset
X_hnc_train, X_hnc_test, y_hnc_train, y_hnc_test = train_test_split(X_hnc_smote, y_hnc_smote, stratify=y_hnc_smote, random_state=0, test_size = 0.2)
# use gridsearchCV for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create parameter dictionary for tuning
hnc_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)
# define the gridsearchCV object with 5 CV
hnc_grid_search_rf = GridSearchCV(rf, param_grid=hnc_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
hnc_grid_search_rf.fit(X_hnc_train, y_hnc_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(hnc_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(hnc_grid_search_rf.best_score_))

In [ ]:
# predict outcome on train set
y_pred_hnc_rf_grid_search_train = hnc_grid_search_rf.predict(X_hnc_train)
# predict probabilities on train set
y_prob_hnc_rf_grid_search_train = hnc_grid_search_rf.predict_proba(X_hnc_train)
# predict outcome on test set
y_pred_hnc_rf_grid_search_test = hnc_grid_search_rf.predict(X_hnc_test)
# predict probabilities on test set
y_prob_hnc_rf_grid_search_test = hnc_grid_search_rf.predict_proba(X_hnc_test)
# print(y_prob_hnc_rf_grid_search_test[:,1])

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_hnc_train, y_pred_hnc_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_hnc_test, y_pred_hnc_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_hnc_rf = confusion_matrix(y_true = y_hnc_test, y_pred = y_pred_hnc_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_hnc_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_hnc_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_hnc_test, y_pred_hnc_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_hnc_rf = hnc_grid_search_rf.predict_proba(X_hnc_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_hnc_test, y_score = y_pred_proba_hnc_rf))
fpr, tpr, thresholds = metrics.roc_curve(y_hnc_test, y_pred_proba_hnc_rf)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# calculate c-index
# convert pred/proba arrays to dataframes and add column labels
X_hnc_test_pred_df = pd.DataFrame([y_pred_hnc_rf_grid_search_test]).transpose()
X_hnc_test_pred_df.columns = ['PredictedOutcome']
X_hnc_test_proba_df = pd.DataFrame([y_prob_hnc_rf_grid_search_test[:,1]]).transpose()
X_hnc_test_proba_df.columns = ['PredictedProbability']

# split dataset to include survival time in X for c-index calculation
X_hnc_cindex = completed_hnc_df.drop(axis=1, columns=['PatientID', 'TwoYearHNCDeath'])
y_hnc_cindex = completed_hnc_df['TwoYearHNCDeath']
# create balanced dataset with survival time
smote = SMOTE(random_state=1, k_neighbors=5)
X_hnc_cindex_smote, y_hnc_cindex_smote = smote.fit_resample(X_hnc_cindex, y_hnc_cindex)
# split the dataset into X and y
X_hnc_train_cindex, X_hnc_test_cindex, y_hnc_train_cindex, y_hnc_test_cindex = train_test_split(X_hnc_cindex_smote, y_hnc_cindex_smote, stratify=y_hnc_cindex_smote, random_state=0, test_size = 0.2)

# append predicted outcome and probability to test set using indexes
hnc_X_test_df = X_hnc_test.copy()
hnc_X_test_df['SurvivalMonths'] = X_hnc_test_cindex['SurvivalMonths']
hnc_X_test_df['TwoYearHNCDeath']= y_hnc_test
hnc_X_test_df.reset_index(inplace=True)
hnc_X_test_df['PredictedOutcome'] = X_hnc_test_pred_df
hnc_X_test_df['PredictedProbability'] = X_hnc_test_proba_df
# print(hnc_X_test_df)

# calculate c-index
hnc_cindex = concordance_index(hnc_X_test_df['SurvivalMonths'], -hnc_X_test_df['PredictedProbability'], hnc_X_test_df['TwoYearHNCDeath']) # the negative value is used for probability, see documentation
print("c-index=", hnc_cindex)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for c-index
hnc_cindex_score = []
n_iterations = 1000
for i in range(n_iterations):
    time, prob, event = resample(X_hnc_test_cindex['SurvivalMonths'], X_hnc_test_proba_df, y_hnc_test, replace=True)
    hnc_ci_score = concordance_index(time, prob, event)
    hnc_cindex_score.append(hnc_ci_score)

# sns.kdeplot(hnc_cindex_score)
# plt.show()

# calculate 95% CI of cindex
hnc_cindex_25_percentile = np.percentile(hnc_cindex_score, 25)
print('25th percentile hnc:', 1-hnc_cindex_25_percentile)
hnc_cindex_50_percentile = np.percentile(hnc_cindex_score, 50) # value should be very close to calculated c-index score
print('50th percentile hnc:', 1-hnc_cindex_50_percentile)
hnc_cindex_75_percentile = np.percentile(hnc_cindex_score, 75)
print('75th percentile hnc:', 1-hnc_cindex_75_percentile)

In [ ]:
# save the model to disk
pickle.dump(hnc_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

# kaplan meier - hnc dataset

In [ ]:
# split dataset with survival time included for merging to dataset with pred and proba without oversampling
X_hnc_surv = completed_hnc_df.drop(axis=1, columns=['PatientID', 'TwoYearHNCDeath'])
y_hnc_surv = completed_hnc_df['TwoYearHNCDeath']
X_hnc_surv_train_time, X_hnc_surv_test_time, y_hnc_surv_train, y_hnc_surv_test = train_test_split(X_hnc_surv, y_hnc_surv, stratify=y_hnc_surv, random_state=0, test_size = 0.2)
# remove survival time from X train and test
X_hnc_surv_train = X_hnc_surv_train_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_hnc_surv_train)
X_hnc_surv_test = X_hnc_surv_test_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_hnc_surv_test)

# use gridsearchCV to find for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]

# create parameter dictionary for tuning
hnc_surv_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)

# define the gridsearchCV object with 5 CV
hnc_surv_grid_search_rf = GridSearchCV(rf, param_grid=hnc_surv_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
hnc_surv_grid_search_rf.fit(X_hnc_surv_train, y_hnc_surv_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(hnc_surv_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(hnc_surv_grid_search_rf.best_score_))

# predict outcome using tuned on train set
y_pred_hnc_surv_rf_grid_search_train = hnc_surv_grid_search_rf.predict(X_hnc_surv_train)
# predict probabilities on train set
y_prob_hnc_surv_rf_grid_search_train = hnc_surv_grid_search_rf.predict_proba(X_hnc_surv_train)
# print(y_prob_hnc_surv_rf_grid_search_train)

# calculate percentiles on train set
hnc_surv_train_25_percentile = np.percentile (y_prob_hnc_surv_rf_grid_search_train[:,1], 25) # for prediction of class 1 'death'
# print('25th percentile:', hnc_surv_train_25_percentile)
hnc_surv_train_50_percentile = np.percentile (y_prob_hnc_surv_rf_grid_search_train[:,1], 50)
# print('50th percentile:', hnc_surv_train_50_percentile)
hnc_surv_train_75_percentile = np.percentile (y_prob_hnc_surv_rf_grid_search_train[:,1], 75)
# print('75th percentile:', hnc_surv_train_75_percentile)

# predict outcome using tuned model on test set
y_pred_hnc_surv_rf_grid_search_test = hnc_surv_grid_search_rf.predict(X_hnc_surv_test)
# print(y_pred_hnc_surv_rf_grid_search_test)
# predict probabilities on test set
y_prob_hnc_surv_rf_grid_search_test = hnc_surv_grid_search_rf.predict_proba(X_hnc_surv_test)
# print(y_prob_hnc_surv_rf_grid_search_test[:,1])

# calculate percentiles on test set
X_hnc_surv_test_25_percentile = np.percentile (y_prob_hnc_surv_rf_grid_search_test[:,1], 25)
# print('25th percentile:', X_hnc_surv_test_25_percentile)
X_hnc_surv_test_50_percentile = np.percentile (y_prob_hnc_surv_rf_grid_search_test[:,1], 50)
# print('50th percentile:', X_hnc_surv_test_50_percentile)
X_hnc_surv_test_75_percentile = np.percentile (y_prob_hnc_surv_rf_grid_search_test[:,1], 75)
# print('75th percentile:', X_hnc_surv_test_75_percentile)

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_hnc_surv_train, y_pred_hnc_surv_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_hnc_surv_test, y_pred_hnc_surv_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_hnc_surv_rf = confusion_matrix(y_true = y_hnc_surv_test, y_pred = y_pred_hnc_surv_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_hnc_surv_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_hnc_surv_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_hnc_surv_test, y_pred_hnc_surv_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_hnc_surv_rf = hnc_surv_grid_search_rf.predict_proba(X_hnc_surv_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_hnc_surv_test, y_score = y_pred_hnc_surv_rf_grid_search_test))
fpr, tpr, thresholds = metrics.roc_curve(y_hnc_surv_test, y_pred_hnc_surv_rf_grid_search_test)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# save the model to disk
pickle.dump(hnc_surv_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

In [ ]:
# convert pred/proba arrays to dataframes and add column labels
X_hnc_surv_test_pred_df = pd.DataFrame([y_pred_hnc_surv_rf_grid_search_test]).transpose()
X_hnc_surv_test_pred_df.columns = ['PredictedOutcome']
X_hnc_surv_test_proba_df = pd.DataFrame([y_prob_hnc_surv_rf_grid_search_test[:,1]]).transpose()
X_hnc_surv_test_proba_df.columns = ['PredictedProbability']

# append predicted outcome and probability to test set using indexes
hnc_km_df = X_hnc_surv_test.copy()
hnc_km_df['SurvivalMonths'] = X_hnc_surv_test_time['SurvivalMonths']
hnc_km_df['TwoYearHNCDeath']= y_hnc_surv_test
hnc_km_df.reset_index(inplace=True)
hnc_km_df['PredictedOutcome'] = X_hnc_surv_test_pred_df
hnc_km_df['PredictedProbability'] = X_hnc_surv_test_proba_df
# print(hnc_km_df)

# create risk group variable, 3=poor prognosis, 2=medium prognosis, 1=good prognosis
hnc_km_df['RiskGroup'] = [3 if x<hnc_surv_train_25_percentile else 1 if x>hnc_surv_train_75_percentile else 2 for x in hnc_km_df['PredictedProbability']]
# print(hnc_km_df)

In [ ]:
# plot km curve for risk group
fig, ax = plt.subplots(figsize=(6,6), dpi=80)
df = hnc_km_df
ix = df['RiskGroup'] == 'riskgroup'

hnc_km_rg1 = KaplanMeierFitter()
ax = hnc_km_rg1.fit(df[df["RiskGroup"] == 1]['SurvivalMonths'], df[df["RiskGroup"] == 1]['TwoYearHNCDeath'], label='poor prognosis').plot_survival_function(ax=ax)

hnc_km_rg2 = KaplanMeierFitter()
ax = hnc_km_rg2.fit(df[df["RiskGroup"] == 2]['SurvivalMonths'], df[df["RiskGroup"] == 2]['TwoYearHNCDeath'], label='medium prognosis').plot_survival_function(ax=ax)

hnc_km_rg3 = KaplanMeierFitter()
ax = hnc_km_rg3.fit(df[df["RiskGroup"] == 3]['SurvivalMonths'], df[df["RiskGroup"] == 3]['TwoYearHNCDeath'], label='good prognosis').plot_survival_function(ax=ax)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(hnc_km_rg1, hnc_km_rg2, hnc_km_rg3, ax=ax)
ax.set_xlabel("Survival time (months)")
ax.set_xlim(0, 24) # set x axis to 24 months
ax.set_ylabel("Survival")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')
plt.tight_layout()

# save km figure
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
# print event table for each risk group
print(hnc_km_rg1.event_table)
print(hnc_km_rg2.event_table)
print(hnc_km_rg3.event_table)

In [ ]:
# calculate log rank statistics
hnc_results = multivariate_logrank_test(hnc_km_df['SurvivalMonths'], hnc_km_df['RiskGroup'], hnc_km_df['TwoYearHNCDeath'])
hnc_results.print_summary()
print("p value =", hnc_results.p_value)        
print("t statistic =", hnc_results.test_statistic) 

In [ ]:
# calculate c-index
hnc_surv_cindex = concordance_index(hnc_km_df['SurvivalMonths'], -hnc_km_df['PredictedProbability'], hnc_km_df['TwoYearHNCDeath'])
print(hnc_surv_cindex)

# prediction - hpc dataset

In [ ]:
# initiate random forest model
rf = RandomForestClassifier(random_state=0)
# split into 80% train and 20% test using oversampled dataset
X_hpc_train, X_hpc_test, y_hpc_train, y_hpc_test = train_test_split(X_hpc_smote, y_hpc_smote, stratify=y_hpc_smote, random_state=0, test_size = 0.2)
# use gridsearchCV for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create parameter dictionary for tuning
hpc_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)
# define the gridsearchCV object with 5 CV
hpc_grid_search_rf = GridSearchCV(rf, param_grid=hpc_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
hpc_grid_search_rf.fit(X_hpc_train, y_hpc_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(hpc_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(hpc_grid_search_rf.best_score_))

In [ ]:
# predict outcome on train set
y_pred_hpc_rf_grid_search_train = hpc_grid_search_rf.predict(X_hpc_train)
# predict probabilities on train set
y_prob_hpc_rf_grid_search_train = hpc_grid_search_rf.predict_proba(X_hpc_train)
# predict outcome on test set
y_pred_hpc_rf_grid_search_test = hpc_grid_search_rf.predict(X_hpc_test)
# predict probabilities on test set
y_prob_hpc_rf_grid_search_test = hpc_grid_search_rf.predict_proba(X_hpc_test)
# print(y_prob_hpc_rf_grid_search_test[:,1])

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_hpc_train, y_pred_hpc_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_hpc_test, y_pred_hpc_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_hpc_rf = confusion_matrix(y_true = y_hpc_test, y_pred = y_pred_hpc_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_hpc_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_hpc_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_hpc_test, y_pred_hpc_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_hpc_rf = hpc_grid_search_rf.predict_proba(X_hpc_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_hpc_test, y_score = y_pred_proba_hpc_rf))
fpr, tpr, thresholds = metrics.roc_curve(y_hpc_test, y_pred_proba_hpc_rf)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# calculate c-index
# convert pred/proba arrays to dataframes and add column labels
X_hpc_test_pred_df = pd.DataFrame([y_pred_hpc_rf_grid_search_test]).transpose()
X_hpc_test_pred_df.columns = ['PredictedOutcome']
X_hpc_test_proba_df = pd.DataFrame([y_prob_hpc_rf_grid_search_test[:,1]]).transpose()
X_hpc_test_proba_df.columns = ['PredictedProbability']

# split dataset to include survival time in X for c-index calculation
X_hpc_cindex = completed_hpc_df.drop(axis=1, columns=['PatientID', 'TwoYearHNCDeath'])
y_hpc_cindex = completed_hpc_df['TwoYearHNCDeath']
# create balanced dataset with survival time
smote = SMOTE(random_state=1, k_neighbors=5)
X_hpc_cindex_smote, y_hpc_cindex_smote = smote.fit_resample(X_hpc_cindex, y_hpc_cindex)
# split the dataset into X and y
X_hpc_train_cindex, X_hpc_test_cindex, y_hpc_train_cindex, y_hpc_test_cindex = train_test_split(X_hpc_cindex_smote, y_hpc_cindex_smote, stratify=y_hpc_cindex_smote, random_state=0, test_size = 0.2)

# append predicted outcome and probability to test set using indexes
hpc_X_test_df = X_hpc_test.copy()
hpc_X_test_df['SurvivalMonths'] = X_hpc_test_cindex['SurvivalMonths']
hpc_X_test_df['TwoYearHNCDeath']= y_hpc_test
hpc_X_test_df.reset_index(inplace=True)
hpc_X_test_df['PredictedOutcome'] = X_hpc_test_pred_df
hpc_X_test_df['PredictedProbability'] = X_hpc_test_proba_df
# print(hpc_X_test_df)

# calculate c-index
hpc_cindex = concordance_index(hpc_X_test_df['SurvivalMonths'], -hpc_X_test_df['PredictedProbability'], hpc_X_test_df['TwoYearHNCDeath']) # the negative value is used for probability, see documentation
print("c-index=", hpc_cindex)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for c-index
hpc_cindex_score = []
n_iterations = 1000
for i in range(n_iterations):
    time, prob, event = resample(X_hpc_test_cindex['SurvivalMonths'], X_hpc_test_proba_df, y_hpc_test, replace=True)
    hpc_ci_score = concordance_index(time, prob, event)
    hpc_cindex_score.append(hpc_ci_score)

# sns.kdeplot(hc_cindex_score)
# plt.show()

# calculate 95% CI of cindex
hpc_cindex_25_percentile = np.percentile(hpc_cindex_score, 25)
print('25th percentile hpc:', 1-hpc_cindex_25_percentile)
hpc_cindex_50_percentile = np.percentile(hpc_cindex_score, 50) # value should be very close to calculated c-index score
print('50th percentile hpc:', 1-hpc_cindex_50_percentile)
hpc_cindex_75_percentile = np.percentile(hpc_cindex_score, 75)
print('75th percentile hpc:', 1-hpc_cindex_75_percentile)

In [ ]:
# save the model to disk
pickle.dump(hpc_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

# kaplan meier - hpc dataset

In [ ]:
# split dataset with survival time included for merging to dataset with pred and proba without oversampling
X_hpc_surv = completed_hpc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_hpc_surv = completed_hpc_df['TwoYearHNCDeath']
X_hpc_surv_train_time, X_hpc_surv_test_time, y_hpc_surv_train, y_hpc_surv_test = train_test_split(X_hpc_surv, y_hpc_surv, stratify=y_hpc_surv, random_state=0, test_size = 0.2)
# remove survival time from X train and test
X_hpc_surv_train = X_hpc_surv_train_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_hpc_surv_train)
X_hpc_surv_test = X_hpc_surv_test_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_hpc_surv_test)

# use gridsearchCV to find for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]

# create parameter dictionary for tuning
hpc_surv_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)

# define the gridsearchCV object with 5 CV
hpc_surv_grid_search_rf = GridSearchCV(rf, param_grid=hpc_surv_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
hpc_surv_grid_search_rf.fit(X_hpc_surv_train, y_hpc_surv_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(hpc_surv_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(hpc_surv_grid_search_rf.best_score_))

# predict outcome using tuned model on train set
y_pred_hpc_surv_rf_grid_search_train = hpc_surv_grid_search_rf.predict(X_hpc_surv_train)
# predict probabilities on train set
y_prob_hpc_surv_rf_grid_search_train = hpc_surv_grid_search_rf.predict_proba(X_hpc_surv_train)
# print(y_prob_hpc_surv_rf_grid_search_train)

# calculate percentiles on train set
hpc_surv_train_25_percentile = np.percentile (y_prob_hpc_surv_rf_grid_search_train[:,1], 25) # for prediction of class 1 'death'
# print('25th percentile:', hpc_surv_train_25_percentile)
hpc_surv_train_50_percentile = np.percentile (y_prob_hpc_surv_rf_grid_search_train[:,1], 50)
# print('50th percentile:', hpc_surv_train_50_percentile)
hpc_surv_train_75_percentile = np.percentile (y_prob_hpc_surv_rf_grid_search_train[:,1], 75)
# print('75th percentile:', hpc_surv_train_75_percentile)

# predict outcome using tuned model on test set
y_pred_hpc_surv_rf_grid_search_test = hpc_surv_grid_search_rf.predict(X_hpc_surv_test)
# print(y_pred_hpc_surv_rf_grid_search_test)
# predict probabilities on test set
y_prob_hpc_surv_rf_grid_search_test = hpc_surv_grid_search_rf.predict_proba(X_hpc_surv_test)
# print(y_prob_hpc_surv_rf_grid_search_test[:,1])

# calculate percentiles on test set
X_hpc_surv_test_25_percentile = np.percentile (y_prob_hpc_surv_rf_grid_search_test[:,1], 25)
# print('25th percentile:', X_hpc_surv_test_25_percentile)
X_hpc_surv_test_50_percentile = np.percentile (y_prob_hpc_surv_rf_grid_search_test[:,1], 50)
# print('50th percentile:', X_hpc_surv_test_50_percentile)
X_hpc_surv_test_75_percentile = np.percentile (y_prob_hpc_surv_rf_grid_search_test[:,1], 75)
# print('75th percentile:', X_hpc_surv_test_75_percentile)

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_hpc_surv_train, y_pred_hpc_surv_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_hpc_surv_test, y_pred_hpc_surv_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_hpc_surv_rf = confusion_matrix(y_true = y_hpc_surv_test, y_pred = y_pred_hpc_surv_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_hpc_surv_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_hpc_surv_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_hpc_surv_test, y_pred_hpc_surv_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_hpc_surv_rf = hpc_surv_grid_search_rf.predict_proba(X_hpc_surv_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_hpc_surv_test, y_score = y_pred_hpc_surv_rf_grid_search_test))
fpr, tpr, thresholds = metrics.roc_curve(y_hpc_surv_test, y_pred_hpc_surv_rf_grid_search_test)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# save the model to disk
pickle.dump(hpc_surv_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

In [ ]:
# convert pred/proba arrays to dataframes and add column labels
X_hpc_surv_test_pred_df = pd.DataFrame([y_pred_hpc_surv_rf_grid_search_test]).transpose()
X_hpc_surv_test_pred_df.columns = ['PredictedOutcome']
X_hpc_surv_test_proba_df = pd.DataFrame([y_prob_hpc_surv_rf_grid_search_test[:,1]]).transpose()
X_hpc_surv_test_proba_df.columns = ['PredictedProbability']

# append predicted outcome and probability to test set using indexes
hpc_km_df = X_hpc_surv_test.copy()
hpc_km_df['SurvivalMonths'] = X_hpc_surv_test_time['SurvivalMonths']
hpc_km_df['TwoYearHNCDeath']= y_hpc_surv_test
hpc_km_df.reset_index(inplace=True)
hpc_km_df['PredictedOutcome'] = X_hpc_surv_test_pred_df
hpc_km_df['PredictedProbability'] = X_hpc_surv_test_proba_df
# print(hpc_km_df)

# create risk group variable, 3=poor prognosis, 2=medium prognosis, 1=good prognosis
hpc_km_df['RiskGroup'] = [3 if x<hpc_surv_train_25_percentile else 1 if x>hpc_surv_train_75_percentile else 2 for x in hpc_km_df['PredictedProbability']]
# print(hpc_km_df)

In [ ]:
# plot km curve for risk group
fig, ax = plt.subplots(figsize=(6,6), dpi=80)
df = hpc_km_df
ix = df['RiskGroup'] == 'riskgroup'

hpc_km_rg1 = KaplanMeierFitter()
ax = hpc_km_rg1.fit(df[df["RiskGroup"] == 1]['SurvivalMonths'], df[df["RiskGroup"] == 1]['TwoYearHNCDeath'], label='poor prognosis').plot_survival_function(ax=ax)

hpc_km_rg2 = KaplanMeierFitter()
ax = hpc_km_rg2.fit(df[df["RiskGroup"] == 2]['SurvivalMonths'], df[df["RiskGroup"] == 2]['TwoYearHNCDeath'], label='medium prognosis').plot_survival_function(ax=ax)

hpc_km_rg3 = KaplanMeierFitter()
ax = hpc_km_rg3.fit(df[df["RiskGroup"] == 3]['SurvivalMonths'], df[df["RiskGroup"] == 3]['TwoYearHNCDeath'], label='good prognosis').plot_survival_function(ax=ax)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(hpc_km_rg1, hpc_km_rg2, hpc_km_rg3, ax=ax)
ax.set_xlabel("Survival time (months)")
ax.set_xlim(0, 24) # set x axis to 24 months
ax.set_ylabel("Survival")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')
plt.tight_layout()

# save km figure
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
# print event table for each risk group
print(hpc_km_rg1.event_table)
print(hpc_km_rg2.event_table)
print(hpc_km_rg3.event_table)

In [ ]:
# calculate log rank statistics
hpc_results = multivariate_logrank_test(hpc_km_df['SurvivalMonths'], hpc_km_df['RiskGroup'], hpc_km_df['TwoYearHNCDeath'])
hpc_results.print_summary()
print("p value =", hpc_results.p_value)        
print("t statistic =", hpc_results.test_statistic) 

In [ ]:
# calculate c-index
hpc_surv_cindex = concordance_index(hpc_km_df['SurvivalMonths'], -hpc_km_df['PredictedProbability'], hpc_km_df['TwoYearHNCDeath'])
print(hpc_surv_cindex)

# prediction - lyx dataset

In [ ]:
# initiate random forest model
rf = RandomForestClassifier(random_state=0)
# split into 80% train and 20% test using oversampled dataset
X_lyx_train, X_lyx_test, y_lyx_train, y_lyx_test = train_test_split(X_lyx_smote, y_lyx_smote, stratify=y_lyx_smote, random_state=0, test_size = 0.2)
# use gridsearchCV for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create parameter dictionary for tuning
lyx_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)
# define the gridsearchCV object with 5 CV
lyx_grid_search_rf = GridSearchCV(rf, param_grid=lyx_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
lyx_grid_search_rf.fit(X_lyx_train, y_lyx_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(lyx_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(lyx_grid_search_rf.best_score_))

In [ ]:
# predict outcome on train set
y_pred_lyx_rf_grid_search_train = lyx_grid_search_rf.predict(X_lyx_train)
# predict probabilities on train set
y_prob_lyx_rf_grid_search_train = lyx_grid_search_rf.predict_proba(X_lyx_train)
# predict outcome on test set
y_pred_lyx_rf_grid_search_test = lyx_grid_search_rf.predict(X_lyx_test)
# predict probabilities on test set
y_prob_lyx_rf_grid_search_test = lyx_grid_search_rf.predict_proba(X_lyx_test)
# print(y_prob_lyx_rf_grid_search_test[:,1])

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_lyx_train, y_pred_lyx_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_lyx_test, y_pred_lyx_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_lyx_rf = confusion_matrix(y_true = y_lyx_test, y_pred = y_pred_lyx_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_lyx_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_lyx_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_lyx_test, y_pred_lyx_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_lyx_rf = lyx_grid_search_rf.predict_proba(X_lyx_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_lyx_test, y_score = y_pred_proba_lyx_rf))
fpr, tpr, thresholds = metrics.roc_curve(y_lyx_test, y_pred_proba_lyx_rf)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# calculate c-index
# convert pred/proba arrays to dataframes and add column labels
X_lyx_test_pred_df = pd.DataFrame([y_pred_lyx_rf_grid_search_test]).transpose()
X_lyx_test_pred_df.columns = ['PredictedOutcome']
X_lyx_test_proba_df = pd.DataFrame([y_prob_lyx_rf_grid_search_test[:,1]]).transpose()
X_lyx_test_proba_df.columns = ['PredictedProbability']

# split dataset to include survival time in X for c-index calculation
X_lyx_cindex = completed_lyx_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_lyx_cindex = completed_lyx_df['TwoYearHNCDeath']
# create balanced dataset with survival time
smote = SMOTE(random_state=1, k_neighbors=5)
X_lyx_cindex_smote, y_lyx_cindex_smote = smote.fit_resample(X_lyx_cindex, y_lyx_cindex)
# split the dataset into X and y
X_lyx_train_cindex, X_lyx_test_cindex, y_lyx_train_cindex, y_lyx_test_cindex = train_test_split(X_lyx_cindex_smote, y_lyx_cindex_smote, stratify=y_lyx_cindex_smote, random_state=0, test_size = 0.2)

# append predicted outcome and probability to test set using indexes
lyx_X_test_df = X_lyx_test.copy()
lyx_X_test_df['SurvivalMonths'] = X_lyx_test_cindex['SurvivalMonths']
lyx_X_test_df['TwoYearHNCDeath']= y_lyx_test
lyx_X_test_df.reset_index(inplace=True)
lyx_X_test_df['PredictedOutcome'] = X_lyx_test_pred_df
lyx_X_test_df['PredictedProbability'] = X_lyx_test_proba_df
# print(lyx_X_test_df)

# calculate c-index
lyx_cindex = concordance_index(lyx_X_test_df['SurvivalMonths'], -lyx_X_test_df['PredictedProbability'], lyx_X_test_df['TwoYearHNCDeath']) # the negative value is used for probability, see documentation
print("c-index=", lyx_cindex)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for c-index
lyx_cindex_score = []
n_iterations = 1000
for i in range(n_iterations):
    time, prob, event = resample(X_lyx_test_cindex['SurvivalMonths'], X_lyx_test_proba_df, y_lyx_test, replace=True)
    lyx_ci_score = concordance_index(time, prob, event)
    lyx_cindex_score.append(lyx_ci_score)

# sns.kdeplot(lyx_cindex_score)
# plt.show()

# calculate 95% CI of cindex
lyx_cindex_25_percentile = np.percentile(lyx_cindex_score, 25)
print('25th percentile lyx:', 1-lyx_cindex_25_percentile)
lyx_cindex_50_percentile = np.percentile(lyx_cindex_score, 50) # value should be very close to calculated c-index score
print('50th percentile lyx:', 1-lyx_cindex_50_percentile)
lyx_cindex_75_percentile = np.percentile(lyx_cindex_score, 75)
print('75th percentile lyx:', 1-lyx_cindex_75_percentile)

In [ ]:
# save the model to disk
pickle.dump(lyx_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

# kaplan meier - lyx dataset

In [ ]:
# split dataset with survival time included for merging to dataset with pred and proba without oversampling
X_lyx_surv = completed_lyx_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_lyx_surv = completed_lyx_df['TwoYearHNCDeath']
X_lyx_surv_train_time, X_lyx_surv_test_time, y_lyx_surv_train, y_lyx_surv_test = train_test_split(X_lyx_surv, y_lyx_surv, stratify=y_lyx_surv, random_state=0, test_size = 0.2)
# remove survival time from X train and test
X_lyx_surv_train = X_lyx_surv_train_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_lyx_surv_train)
X_lyx_surv_test = X_lyx_surv_test_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_lyx_surv_test)

# use gridsearchCV to find for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]

# create parameter dictionary for tuning
lyx_surv_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)

# define the gridsearchCV object with 5 CV
lyx_surv_grid_search_rf = GridSearchCV(rf, param_grid=lyx_surv_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
lyx_surv_grid_search_rf.fit(X_lyx_surv_train, y_lyx_surv_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(lyx_surv_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(lyx_surv_grid_search_rf.best_score_))

# predict outcome using tuned model on train set
y_pred_lyx_surv_rf_grid_search_train = lyx_surv_grid_search_rf.predict(X_lyx_surv_train)
# predict probabilities on train set
y_prob_lyx_surv_rf_grid_search_train = lyx_surv_grid_search_rf.predict_proba(X_lyx_surv_train)
# print(y_prob_lyx_surv_rf_grid_search_train)

# calculate percentiles on train set
lyx_surv_train_25_percentile = np.percentile (y_prob_lyx_surv_rf_grid_search_train[:,1], 25) # for prediction of class 1 'death'
# print('25th percentile:', lyx_surv_train_25_percentile)
lyx_surv_train_50_percentile = np.percentile (y_prob_lyx_surv_rf_grid_search_train[:,1], 50)
# print('50th percentile:', lyx_surv_train_50_percentile)
lyx_surv_train_75_percentile = np.percentile (y_prob_lyx_surv_rf_grid_search_train[:,1], 75)
# print('75th percentile:', lyx_surv_train_75_percentile)

# predict outcome using tuned model on test set
y_pred_lyx_surv_rf_grid_search_test = lyx_surv_grid_search_rf.predict(X_lyx_surv_test)
# print(y_pred_lyx_surv_rf_grid_search_test)
# predict probabilities on test set
y_prob_lyx_surv_rf_grid_search_test = lyx_surv_grid_search_rf.predict_proba(X_lyx_surv_test)
# print(y_prob_lyx_surv_rf_grid_search_test[:,1])

# calculate percentiles on test set
X_lyx_surv_test_25_percentile = np.percentile (y_prob_lyx_surv_rf_grid_search_test[:,1], 25)
# print('25th percentile:', X_lyx_surv_test_25_percentile)
X_lyx_surv_test_50_percentile = np.percentile (y_prob_lyx_surv_rf_grid_search_test[:,1], 50)
# print('50th percentile:', X_lyx_surv_test_50_percentile)
X_lyx_surv_test_75_percentile = np.percentile (y_prob_lyx_surv_rf_grid_search_test[:,1], 75)
# print('75th percentile:', X_lyx_surv_test_75_percentile)

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_lyx_surv_train, y_pred_lyx_surv_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_lyx_surv_test, y_pred_lyx_surv_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_lyx_surv_rf = confusion_matrix(y_true = y_lyx_surv_test, y_pred = y_pred_lyx_surv_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_lyx_surv_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_lyx_surv_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_lyx_surv_test, y_pred_lyx_surv_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_lyx_surv_rf = lyx_surv_grid_search_rf.predict_proba(X_lyx_surv_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_lyx_surv_test, y_score = y_pred_lyx_surv_rf_grid_search_test))
fpr, tpr, thresholds = metrics.roc_curve(y_lyx_surv_test, y_pred_lyx_surv_rf_grid_search_test)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# save the model to disk
pickle.dump(lyx_surv_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

In [ ]:
# convert pred/proba arrays to dataframes and add column labels
X_lyx_surv_test_pred_df = pd.DataFrame([y_pred_lyx_surv_rf_grid_search_test]).transpose()
X_lyx_surv_test_pred_df.columns = ['PredictedOutcome']
X_lyx_surv_test_proba_df = pd.DataFrame([y_prob_lyx_surv_rf_grid_search_test[:,1]]).transpose()
X_lyx_surv_test_proba_df.columns = ['PredictedProbability']

# append predicted outcome and probability to test set using indexes
lyx_km_df = X_lyx_surv_test.copy()
lyx_km_df['SurvivalMonths'] = X_lyx_surv_test_time['SurvivalMonths']
lyx_km_df['TwoYearHNCDeath']= y_lyx_surv_test
lyx_km_df.reset_index(inplace=True)
lyx_km_df['PredictedOutcome'] = X_lyx_surv_test_pred_df
lyx_km_df['PredictedProbability'] = X_lyx_surv_test_proba_df
# print(lyx_km_df)

# create risk group variable, 3=poor prognosis, 2=medium prognosis, 1=good prognosis
lyx_km_df['RiskGroup'] = [3 if x<lyx_surv_train_25_percentile else 1 if x>lyx_surv_train_75_percentile else 2 for x in lyx_km_df['PredictedProbability']]
# print(lyx_km_df)

In [ ]:
# plot km curve for risk group
fig, ax = plt.subplots(figsize=(6,6), dpi=80)
df = lyx_km_df
ix = df['RiskGroup'] == 'riskgroup'

lyx_km_rg1 = KaplanMeierFitter()
ax = lyx_km_rg1.fit(df[df["RiskGroup"] == 1]['SurvivalMonths'], df[df["RiskGroup"] == 1]['TwoYearHNCDeath'], label='poor prognosis').plot_survival_function(ax=ax)

lyx_km_rg2 = KaplanMeierFitter()
ax = lyx_km_rg2.fit(df[df["RiskGroup"] == 2]['SurvivalMonths'], df[df["RiskGroup"] == 2]['TwoYearHNCDeath'], label='medium prognosis').plot_survival_function(ax=ax)

lyx_km_rg3 = KaplanMeierFitter()
ax = lyx_km_rg3.fit(df[df["RiskGroup"] == 3]['SurvivalMonths'], df[df["RiskGroup"] == 3]['TwoYearHNCDeath'], label='good prognosis').plot_survival_function(ax=ax)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(lyx_km_rg1, lyx_km_rg2, lyx_km_rg3, ax=ax)
ax.set_xlabel("Survival time (months)")
ax.set_xlim(0, 24) # set x axis to 24 months
ax.set_ylabel("Survival")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')
plt.tight_layout()

# save km figure
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
# print event table for each risk group
print(lyx_km_rg1.event_table)
print(lyx_km_rg2.event_table)
print(lyx_km_rg3.event_table)

In [ ]:
# calculate log rank statistics
lyx_results = multivariate_logrank_test(lyx_km_df['SurvivalMonths'], lyx_km_df['RiskGroup'], lyx_km_df['TwoYearHNCDeath'])
lyx_results.print_summary()
print("p value =", lyx_results.p_value)        
print("t statistic =", lyx_results.test_statistic) 

In [ ]:
# calculate c-index
lyx_surv_cindex = concordance_index(lyx_km_df['SurvivalMonths'], -lyx_km_df['PredictedProbability'], lyx_km_df['TwoYearHNCDeath'])
print(lyx_surv_cindex)

# prediction - npc dataset

In [ ]:
# initiate random forest model
rf = RandomForestClassifier(random_state=0)
# split into 80% train and 20% test using oversampled dataset
X_npc_train, X_npc_test, y_npc_train, y_npc_test = train_test_split(X_npc_smote, y_npc_smote, stratify=y_npc_smote, random_state=0, test_size = 0.2)
# use gridsearchCV for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create parameter dictionary for tuning
npc_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)
# define the gridsearchCV object with 5 CV
npc_grid_search_rf = GridSearchCV(rf, param_grid=npc_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
npc_grid_search_rf.fit(X_npc_train, y_npc_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(npc_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(npc_grid_search_rf.best_score_))

In [ ]:
# predict outcome on train set
y_pred_npc_rf_grid_search_train = npc_grid_search_rf.predict(X_npc_train)
# predict probabilities on train set
y_prob_npc_rf_grid_search_train = npc_grid_search_rf.predict_proba(X_npc_train)
# predict outcome on test set
y_pred_npc_rf_grid_search_test = npc_grid_search_rf.predict(X_npc_test)
# predict probabilities on test set
y_prob_npc_rf_grid_search_test = npc_grid_search_rf.predict_proba(X_npc_test)
# print(y_prob_npc_rf_grid_search_test[:,1])

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_npc_train, y_pred_npc_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_npc_test, y_pred_npc_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_npc_rf = confusion_matrix(y_true = y_npc_test, y_pred = y_pred_npc_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_npc_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_npc_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_npc_test, y_pred_npc_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_npc_rf = npc_grid_search_rf.predict_proba(X_npc_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_npc_test, y_score = y_pred_proba_npc_rf))
fpr, tpr, thresholds = metrics.roc_curve(y_npc_test, y_pred_proba_npc_rf)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# calculate c-index
# convert pred/proba arrays to dataframes and add column labels
X_npc_test_pred_df = pd.DataFrame([y_pred_npc_rf_grid_search_test]).transpose()
X_npc_test_pred_df.columns = ['PredictedOutcome']
X_npc_test_proba_df = pd.DataFrame([y_prob_npc_rf_grid_search_test[:,1]]).transpose()
X_npc_test_proba_df.columns = ['PredictedProbability']

# split dataset to include survival time in X for c-index calculation
X_npc_cindex = completed_npc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_npc_cindex = completed_npc_df['TwoYearHNCDeath']
# create balanced dataset with survival time
smote = SMOTE(random_state=1, k_neighbors=5)
X_npc_cindex_smote, y_npc_cindex_smote = smote.fit_resample(X_npc_cindex, y_npc_cindex)
# split the dataset into X and y
X_npc_train_cindex, X_npc_test_cindex, y_npc_train_cindex, y_npc_test_cindex = train_test_split(X_npc_cindex_smote, y_npc_cindex_smote, stratify=y_npc_cindex_smote, random_state=0, test_size = 0.2)

# append predicted outcome and probability to test set using indexes
npc_X_test_df = X_npc_test.copy()
npc_X_test_df['SurvivalMonths'] = X_npc_test_cindex['SurvivalMonths']
npc_X_test_df['TwoYearHNCDeath']= y_npc_test
npc_X_test_df.reset_index(inplace=True)
npc_X_test_df['PredictedOutcome'] = X_npc_test_pred_df
npc_X_test_df['PredictedProbability'] = X_npc_test_proba_df
# print(npc_X_test_df)

# calculate c-index
npc_cindex = concordance_index(npc_X_test_df['SurvivalMonths'], -npc_X_test_df['PredictedProbability'], npc_X_test_df['TwoYearHNCDeath']) # the negative value is used for probability, see documentation
print("c-index=", npc_cindex)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for c-index
npc_cindex_score = []
n_iterations = 1000
for i in range(n_iterations):
    time, prob, event = resample(X_npc_test_cindex['SurvivalMonths'], X_npc_test_proba_df, y_npc_test, replace=True)
    npc_ci_score = concordance_index(time, prob, event)
    npc_cindex_score.append(npc_ci_score)

# sns.kdeplot(npc_cindex_score)
# plt.show()

# calculate 95% CI of cindex
npc_cindex_25_percentile = np.percentile(npc_cindex_score, 25)
print('25th percentile npc:', 1-npc_cindex_25_percentile)
npc_cindex_50_percentile = np.percentile(npc_cindex_score, 50) # value should be very close to calculated c-index score
print('50th percentile npc:', 1-npc_cindex_50_percentile)
npc_cindex_75_percentile = np.percentile(npc_cindex_score, 75)
print('75th percentile npc:', 1-npc_cindex_75_percentile)

In [ ]:
# save the model to disk
pickle.dump(npc_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

# kaplan meier - npc dataset

In [ ]:
# split dataset with survival time included for merging to dataset with pred and proba without oversampling
X_npc_surv = completed_npc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_npc_surv = completed_npc_df['TwoYearHNCDeath']
X_npc_surv_train_time, X_npc_surv_test_time, y_npc_surv_train, y_npc_surv_test = train_test_split(X_npc_surv, y_npc_surv, stratify=y_npc_surv, random_state=0, test_size = 0.2)
# remove survival time from X train and test
X_npc_surv_train = X_npc_surv_train_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_npc_surv_train)
X_npc_surv_test = X_npc_surv_test_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_npc_surv_test)

# use gridsearchCV to find for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]

# create parameter dictionary for tuning
npc_surv_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)

# define the gridsearchCV object with 5 CV
npc_surv_grid_search_rf = GridSearchCV(rf, param_grid=npc_surv_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
npc_surv_grid_search_rf.fit(X_npc_surv_train, y_npc_surv_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(npc_surv_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(npc_surv_grid_search_rf.best_score_))

# predict outcome using tuned model on train set
y_pred_npc_surv_rf_grid_search_train = npc_surv_grid_search_rf.predict(X_npc_surv_train)
# predict probabilities on train set
y_prob_npc_surv_rf_grid_search_train = npc_surv_grid_search_rf.predict_proba(X_npc_surv_train)
# print(y_prob_npc_surv_rf_grid_search_train)

# calculate percentiles on train set
npc_surv_train_25_percentile = np.percentile (y_prob_npc_surv_rf_grid_search_train[:,1], 25) # for prediction of class 1 'death'
# print('25th percentile:', npc_surv_train_25_percentile)
npc_surv_train_50_percentile = np.percentile (y_prob_npc_surv_rf_grid_search_train[:,1], 50)
# print('50th percentile:', npc_surv_train_50_percentile)
npc_surv_train_75_percentile = np.percentile (y_prob_npc_surv_rf_grid_search_train[:,1], 75)
# print('75th percentile:', npc_surv_train_75_percentile)

# predict outcome using tuned model on test set
y_pred_npc_surv_rf_grid_search_test = npc_surv_grid_search_rf.predict(X_npc_surv_test)
# print(y_pred_npc_surv_rf_grid_search_test)
# predict probabilities on test set
y_prob_npc_surv_rf_grid_search_test = npc_surv_grid_search_rf.predict_proba(X_npc_surv_test)
# print(y_prob_npc_surv_rf_grid_search_test[:,1])

# calculate percentiles on test set
X_npc_surv_test_25_percentile = np.percentile (y_prob_npc_surv_rf_grid_search_test[:,1], 25)
# print('25th percentile:', X_npc_surv_test_25_percentile)
X_npc_surv_test_50_percentile = np.percentile (y_prob_npc_surv_rf_grid_search_test[:,1], 50)
# print('50th percentile:', X_npc_surv_test_50_percentile)
X_npc_surv_test_75_percentile = np.percentile (y_prob_npc_surv_rf_grid_search_test[:,1], 75)
# print('75th percentile:', X_npc_surv_test_75_percentile)

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_npc_surv_train, y_pred_npc_surv_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_npc_surv_test, y_pred_npc_surv_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_npc_surv_rf = confusion_matrix(y_true = y_npc_surv_test, y_pred = y_pred_npc_surv_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_npc_surv_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_npc_surv_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_npc_surv_test, y_pred_npc_surv_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_npc_surv_rf = npc_surv_grid_search_rf.predict_proba(X_npc_surv_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_npc_surv_test, y_score = y_pred_npc_surv_rf_grid_search_test))
fpr, tpr, thresholds = metrics.roc_curve(y_npc_surv_test, y_pred_npc_surv_rf_grid_search_test)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# save the model to disk
pickle.dump(npc_surv_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

In [ ]:
# convert pred/proba arrays to dataframes and add column labels
X_npc_surv_test_pred_df = pd.DataFrame([y_pred_npc_surv_rf_grid_search_test]).transpose()
X_npc_surv_test_pred_df.columns = ['PredictedOutcome']
X_npc_surv_test_proba_df = pd.DataFrame([y_prob_npc_surv_rf_grid_search_test[:,1]]).transpose()
X_npc_surv_test_proba_df.columns = ['PredictedProbability']

# append predicted outcome and probability to test set using indexes
npc_km_df = X_npc_surv_test.copy()
npc_km_df['SurvivalMonths'] = X_npc_surv_test_time['SurvivalMonths']
npc_km_df['TwoYearHNCDeath']= y_npc_surv_test
npc_km_df.reset_index(inplace=True)
npc_km_df['PredictedOutcome'] = X_npc_surv_test_pred_df
npc_km_df['PredictedProbability'] = X_npc_surv_test_proba_df
# print(npc_km_df)

# create risk group variable, 3=poor prognosis, 2=medium prognosis, 1=good prognosis
npc_km_df['RiskGroup'] = [3 if x<npc_surv_train_25_percentile else 1 if x>npc_surv_train_75_percentile else 2 for x in npc_km_df['PredictedProbability']]
# print(npc_km_df)

In [ ]:
# plot km curve for risk group
fig, ax = plt.subplots(figsize=(6,6), dpi=80)
df = npc_km_df
ix = df['RiskGroup'] == 'riskgroup'

npc_km_rg1 = KaplanMeierFitter()
ax = npc_km_rg1.fit(df[df["RiskGroup"] == 1]['SurvivalMonths'], df[df["RiskGroup"] == 1]['TwoYearHNCDeath'], label='poor prognosis').plot_survival_function(ax=ax)

npc_km_rg2 = KaplanMeierFitter()
ax = npc_km_rg2.fit(df[df["RiskGroup"] == 2]['SurvivalMonths'], df[df["RiskGroup"] == 2]['TwoYearHNCDeath'], label='medium prognosis').plot_survival_function(ax=ax)

# npc_km_rg3 = KaplanMeierFitter()
# ax = npc_km_rg3.fit(df[df["RiskGroup"] == 3]['SurvivalMonths'], df[df["RiskGroup"] == 3]['TwoYearHNCDeath'], label='good prognosis').plot_survival_function(ax=ax)

from lifelines.plotting import add_at_risk_counts
# add_at_risk_counts(npc_km_rg1, npc_km_rg2, npc_km_rg3, ax=ax)
add_at_risk_counts(npc_km_rg1, npc_km_rg2, ax=ax)
ax.set_xlabel("Survival time (months)")
ax.set_xlim(0, 24) # set x axis to 24 months
ax.set_ylabel("Survival")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')
plt.tight_layout()

# save km figure
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
# print event table for each risk group
print(npc_km_rg1.event_table)
print(npc_km_rg2.event_table)
# print(npc_km_rg3.event_table)

In [ ]:
# calculate log rank statistics
npc_results = multivariate_logrank_test(npc_km_df['SurvivalMonths'], npc_km_df['RiskGroup'], npc_km_df['TwoYearHNCDeath'])
npc_results.print_summary()
print("p value =", npc_results.p_value)        
print("t statistic =", npc_results.test_statistic) 

In [ ]:
# calculate c-index
npc_surv_cindex = concordance_index(npc_km_df['SurvivalMonths'], -npc_km_df['PredictedProbability'], npc_km_df['TwoYearHNCDeath'])
print(npc_surv_cindex)

# prediction - occ dataset

In [ ]:
# initiate random forest model
rf = RandomForestClassifier(random_state=0)
# split into 80% train and 20% test using oversampled dataset
X_occ_train, X_occ_test, y_occ_train, y_occ_test = train_test_split(X_occ_smote, y_occ_smote, stratify=y_occ_smote, random_state=0, test_size = 0.2)
# use gridsearchCV for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create parameter dictionary for tuning
occ_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)
# define the gridsearchCV object with 5 CV
occ_grid_search_rf = GridSearchCV(rf, param_grid=occ_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
occ_grid_search_rf.fit(X_occ_train, y_occ_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(occ_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(occ_grid_search_rf.best_score_))

In [ ]:
# predict outcome on train set
y_pred_occ_rf_grid_search_train = occ_grid_search_rf.predict(X_occ_train)
# predict probabilities on train set
y_prob_occ_rf_grid_search_train = occ_grid_search_rf.predict_proba(X_occ_train)
# predict outcome on test set
y_pred_occ_rf_grid_search_test = occ_grid_search_rf.predict(X_occ_test)
# predict probabilities on test set
y_prob_occ_rf_grid_search_test = occ_grid_search_rf.predict_proba(X_occ_test)
# print(y_prob_occ_rf_grid_search_test[:,1])

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_occ_train, y_pred_occ_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_occ_test, y_pred_occ_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_occ_rf = confusion_matrix(y_true = y_occ_test, y_pred = y_pred_occ_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_occ_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_occ_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_occ_test, y_pred_occ_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_occ_rf = occ_grid_search_rf.predict_proba(X_occ_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_occ_test, y_score = y_pred_proba_occ_rf))
fpr, tpr, thresholds = metrics.roc_curve(y_occ_test, y_pred_proba_occ_rf)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# calculate c-index
# convert pred/proba arrays to dataframes and add column labels
X_occ_test_pred_df = pd.DataFrame([y_pred_occ_rf_grid_search_test]).transpose()
X_occ_test_pred_df.columns = ['PredictedOutcome']
X_occ_test_proba_df = pd.DataFrame([y_prob_occ_rf_grid_search_test[:,1]]).transpose()
X_occ_test_proba_df.columns = ['PredictedProbability']

# split dataset to include survival time in X for c-index calculation
X_occ_cindex = completed_occ_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_occ_cindex = completed_occ_df['TwoYearHNCDeath']
# create balanced dataset with survival time
smote = SMOTE(random_state=1, k_neighbors=5)
X_occ_cindex_smote, y_occ_cindex_smote = smote.fit_resample(X_occ_cindex, y_occ_cindex)
# split the dataset into X and y
X_occ_train_cindex, X_occ_test_cindex, y_occ_train_cindex, y_occ_test_cindex = train_test_split(X_occ_cindex_smote, y_occ_cindex_smote, stratify=y_occ_cindex_smote, random_state=0, test_size = 0.2)

# append predicted outcome and probability to test set using indexes
occ_X_test_df = X_occ_test.copy()
occ_X_test_df['SurvivalMonths'] = X_occ_test_cindex['SurvivalMonths']
occ_X_test_df['TwoYearHNCDeath']= y_occ_test
occ_X_test_df.reset_index(inplace=True)
occ_X_test_df['PredictedOutcome'] = X_occ_test_pred_df
occ_X_test_df['PredictedProbability'] = X_occ_test_proba_df
# print(occ_X_test_df)

# calculate c-index
occ_cindex = concordance_index(occ_X_test_df['SurvivalMonths'], -occ_X_test_df['PredictedProbability'], occ_X_test_df['TwoYearHNCDeath']) # the negative value is used for probability, see documentation
print("c-index=", occ_cindex)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for c-index
occ_cindex_score = []
n_iterations = 1000
for i in range(n_iterations):
    time, prob, event = resample(X_occ_test_cindex['SurvivalMonths'], X_occ_test_proba_df, y_occ_test, replace=True)
    occ_ci_score = concordance_index(time, prob, event)
    occ_cindex_score.append(occ_ci_score)

# sns.kdeplot(occ_cindex_score)
# plt.show()

# calculate 95% CI of cindex
occ_cindex_25_percentile = np.percentile(occ_cindex_score, 25)
print('25th percentile occ:', 1-occ_cindex_25_percentile)
occ_cindex_50_percentile = np.percentile(occ_cindex_score, 50) # value should be very close to calculated c-index score
print('50th percentile occ:', 1-occ_cindex_50_percentile)
occ_cindex_75_percentile = np.percentile(occ_cindex_score, 75)
print('75th percentile occ:', 1-occ_cindex_75_percentile)

In [ ]:
# save the model to disk
pickle.dump(occ_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

# kaplan meier - occ dataset

In [ ]:
# split dataset with survival time included for merging to dataset with pred and proba without oversampling
X_occ_surv = completed_occ_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_occ_surv = completed_occ_df['TwoYearHNCDeath']
X_occ_surv_train_time, X_occ_surv_test_time, y_occ_surv_train, y_occ_surv_test = train_test_split(X_occ_surv, y_occ_surv, stratify=y_occ_surv, random_state=0, test_size = 0.2)
# remove survival time from X train and test
X_occ_surv_train = X_occ_surv_train_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_occ_surv_train)
X_occ_surv_test = X_occ_surv_test_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_occ_surv_test)

# use gridsearchCV to find for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]

# create parameter dictionary for tuning
occ_surv_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)

# define the gridsearchCV object with 5 CV
occ_surv_grid_search_rf = GridSearchCV(rf, param_grid=occ_surv_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
occ_surv_grid_search_rf.fit(X_occ_surv_train, y_occ_surv_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(occ_surv_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(occ_surv_grid_search_rf.best_score_))

# predict outcome using tuned model on train set
y_pred_occ_surv_rf_grid_search_train = occ_surv_grid_search_rf.predict(X_occ_surv_train)
# predict probabilities on train set
y_prob_occ_surv_rf_grid_search_train = occ_surv_grid_search_rf.predict_proba(X_occ_surv_train)
# print(y_prob_occ_surv_rf_grid_search_train)

# calculate percentiles on train set
occ_surv_train_25_percentile = np.percentile (y_prob_occ_surv_rf_grid_search_train[:,1], 25) # for prediction of class 1 'death'
# print('25th percentile:', occ_surv_train_25_percentile)
occ_surv_train_50_percentile = np.percentile (y_prob_occ_surv_rf_grid_search_train[:,1], 50)
# print('50th percentile:', occ_surv_train_50_percentile)
occ_surv_train_75_percentile = np.percentile (y_prob_occ_surv_rf_grid_search_train[:,1], 75)
# print('75th percentile:', occ_surv_train_75_percentile)

# predict outcome using tuned model on test set
y_pred_occ_surv_rf_grid_search_test = occ_surv_grid_search_rf.predict(X_occ_surv_test)
# print(y_pred_occ_surv_rf_grid_search_test)
# predict probabilities on test set
y_prob_occ_surv_rf_grid_search_test = occ_surv_grid_search_rf.predict_proba(X_occ_surv_test)
# print(y_prob_occ_surv_rf_grid_search_test[:,1])

# calculate percentiles on test set
X_occ_surv_test_25_percentile = np.percentile (y_prob_occ_surv_rf_grid_search_test[:,1], 25)
# print('25th percentile:', X_occ_surv_test_25_percentile)
X_occ_surv_test_50_percentile = np.percentile (y_prob_occ_surv_rf_grid_search_test[:,1], 50)
# print('50th percentile:', X_occ_surv_test_50_percentile)
X_occ_surv_test_75_percentile = np.percentile (y_prob_occ_surv_rf_grid_search_test[:,1], 75)
# print('75th percentile:', X_occ_surv_test_75_percentile)

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_occ_surv_train, y_pred_occ_surv_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_occ_surv_test, y_pred_occ_surv_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_occ_surv_rf = confusion_matrix(y_true = y_occ_surv_test, y_pred = y_pred_occ_surv_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_occ_surv_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_occ_surv_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_occ_surv_test, y_pred_occ_surv_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_occ_surv_rf = occ_surv_grid_search_rf.predict_proba(X_occ_surv_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_occ_surv_test, y_score = y_pred_occ_surv_rf_grid_search_test))
fpr, tpr, thresholds = metrics.roc_curve(y_occ_surv_test, y_pred_occ_surv_rf_grid_search_test)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# save the model to disk
pickle.dump(occ_surv_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

In [ ]:
# convert pred/proba arrays to dataframes and add column labels
X_occ_surv_test_pred_df = pd.DataFrame([y_pred_occ_surv_rf_grid_search_test]).transpose()
X_occ_surv_test_pred_df.columns = ['PredictedOutcome']
X_occ_surv_test_proba_df = pd.DataFrame([y_prob_occ_surv_rf_grid_search_test[:,1]]).transpose()
X_occ_surv_test_proba_df.columns = ['PredictedProbability']

# append predicted outcome and probability to test set using indexes
occ_km_df = X_occ_surv_test.copy()
occ_km_df['SurvivalMonths'] = X_occ_surv_test_time['SurvivalMonths']
occ_km_df['TwoYearHNCDeath']= y_occ_surv_test
occ_km_df.reset_index(inplace=True)
occ_km_df['PredictedOutcome'] = X_occ_surv_test_pred_df
occ_km_df['PredictedProbability'] = X_occ_surv_test_proba_df
# print(occ_km_df)

# create risk group variable, 3=poor prognosis, 2=medium prognosis, 1=good prognosis
occ_km_df['RiskGroup'] = [3 if x<occ_surv_train_25_percentile else 1 if x>occ_surv_train_75_percentile else 2 for x in occ_km_df['PredictedProbability']]
# print(occ_km_df)

In [ ]:
# plot km curve for risk group
fig, ax = plt.subplots(figsize=(6,6), dpi=80)
df = occ_km_df
ix = df['RiskGroup'] == 'riskgroup'

occ_km_rg1 = KaplanMeierFitter()
ax = occ_km_rg1.fit(df[df["RiskGroup"] == 1]['SurvivalMonths'], df[df["RiskGroup"] == 1]['TwoYearHNCDeath'], label='poor prognosis').plot_survival_function(ax=ax)

occ_km_rg2 = KaplanMeierFitter()
ax = occ_km_rg2.fit(df[df["RiskGroup"] == 2]['SurvivalMonths'], df[df["RiskGroup"] == 2]['TwoYearHNCDeath'], label='medium prognosis').plot_survival_function(ax=ax)

occ_km_rg3 = KaplanMeierFitter()
ax = occ_km_rg3.fit(df[df["RiskGroup"] == 3]['SurvivalMonths'], df[df["RiskGroup"] == 3]['TwoYearHNCDeath'], label='good prognosis').plot_survival_function(ax=ax)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(occ_km_rg1, occ_km_rg2, occ_km_rg3, ax=ax)
ax.set_xlabel("Survival time (months)")
ax.set_xlim(0, 24) # set x axis to 24 months
ax.set_ylabel("Survival")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')
plt.tight_layout()

# save km figure
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
# print event table for each risk group
print(occ_km_rg1.event_table)
print(occ_km_rg2.event_table)
print(occ_km_rg3.event_table)

In [ ]:
# calculate log rank statistics
occ_results = multivariate_logrank_test(occ_km_df['SurvivalMonths'], occ_km_df['RiskGroup'], occ_km_df['TwoYearHNCDeath'])
occ_results.print_summary()
print("p value =", occ_results.p_value)        
print("t statistic =", occ_results.test_statistic) 

In [ ]:
# calculate c-index
occ_surv_cindex = concordance_index(occ_km_df['SurvivalMonths'], -occ_km_df['PredictedProbability'], occ_km_df['TwoYearHNCDeath'])
print(occ_surv_cindex)

# prediction - opc dataset

In [ ]:
# initiate random forest model
rf = RandomForestClassifier(random_state=0)
# split into 80% train and 20% test using oversampled dataset
X_opc_train, X_opc_test, y_opc_train, y_opc_test = train_test_split(X_opc_smote, y_opc_smote, stratify=y_opc_smote, random_state=0, test_size = 0.2)
# use gridsearchCV for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]
# create parameter dictionary for tuning
opc_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)
# define the gridsearchCV object with 5 CV
opc_grid_search_rf = GridSearchCV(rf, param_grid=opc_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
opc_grid_search_rf.fit(X_opc_train, y_opc_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(opc_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(opc_grid_search_rf.best_score_))

In [ ]:
# predict outcome on train set
y_pred_opc_rf_grid_search_train = opc_grid_search_rf.predict(X_opc_train)
# predict probabilities on train set
y_prob_opc_rf_grid_search_train = opc_grid_search_rf.predict_proba(X_opc_train)
# predict outcome on test set
y_pred_opc_rf_grid_search_test = opc_grid_search_rf.predict(X_opc_test)
# predict probabilities on test set
y_prob_opc_rf_grid_search_test = opc_grid_search_rf.predict_proba(X_opc_test)
# print(y_prob_opc_rf_grid_search_test[:,1])

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_opc_train, y_pred_opc_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_opc_test, y_pred_opc_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_opc_rf = confusion_matrix(y_true = y_opc_test, y_pred = y_pred_opc_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_opc_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_opc_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_opc_test, y_pred_opc_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_opc_rf = opc_grid_search_rf.predict_proba(X_opc_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_opc_test, y_score = y_pred_proba_opc_rf))
fpr, tpr, thresholds = metrics.roc_curve(y_opc_test, y_pred_proba_opc_rf)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# calculate c-index
# convert pred/proba arrays to dataframes and add column labels
X_opc_test_pred_df = pd.DataFrame([y_pred_opc_rf_grid_search_test]).transpose()
X_opc_test_pred_df.columns = ['PredictedOutcome']
X_opc_test_proba_df = pd.DataFrame([y_prob_opc_rf_grid_search_test[:,1]]).transpose()
X_opc_test_proba_df.columns = ['PredictedProbability']

# split dataset to include survival time in X for c-index calculation
X_opc_cindex = completed_opc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_opc_cindex = completed_opc_df['TwoYearHNCDeath']
# create balanced dataset with survival time
smote = SMOTE(random_state=1, k_neighbors=5)
X_opc_cindex_smote, y_opc_cindex_smote = smote.fit_resample(X_opc_cindex, y_opc_cindex)
# split the dataset into X and y
X_opc_train_cindex, X_opc_test_cindex, y_opc_train_cindex, y_opc_test_cindex = train_test_split(X_opc_cindex_smote, y_opc_cindex_smote, stratify=y_opc_cindex_smote, random_state=0, test_size = 0.2)

# append predicted outcome and probability to test set using indexes
opc_X_test_df = X_opc_test.copy()
opc_X_test_df['SurvivalMonths'] = X_opc_test_cindex['SurvivalMonths']
opc_X_test_df['TwoYearHNCDeath']= y_opc_test
opc_X_test_df.reset_index(inplace=True)
opc_X_test_df['PredictedOutcome'] = X_opc_test_pred_df
opc_X_test_df['PredictedProbability'] = X_opc_test_proba_df
# print(opc_X_test_df)

# calculate c-index
opc_cindex = concordance_index(opc_X_test_df['SurvivalMonths'], -opc_X_test_df['PredictedProbability'], opc_X_test_df['TwoYearHNCDeath']) # the negative value is used for probability, see documentation
print("c-index=", opc_cindex)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for c-index
opc_cindex_score = []
n_iterations = 1000
for i in range(n_iterations):
    time, prob, event = resample(X_opc_test_cindex['SurvivalMonths'], X_opc_test_proba_df, y_opc_test, replace=True)
    opc_ci_score = concordance_index(time, prob, event)
    opc_cindex_score.append(opc_ci_score)

# sns.kdeplot(opc_cindex_score)
# plt.show()

# calculate 95% CI of cindex
opc_cindex_25_percentile = np.percentile(opc_cindex_score, 25)
print('25th percentile opc:', 1-opc_cindex_25_percentile)
opc_cindex_50_percentile = np.percentile(opc_cindex_score, 50) # value should be very close to calculated c-index score
print('50th percentile opc:', 1-opc_cindex_50_percentile)
opc_cindex_75_percentile = np.percentile(opc_cindex_score, 75)
print('75th percentile opc:', 1-opc_cindex_75_percentile)

In [ ]:
# save the model to disk
pickle.dump(opc_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

# kaplan meier - opc dataset

In [ ]:
# split dataset with survival time included for merging to dataset with pred and proba without oversampling
X_opc_surv = completed_opc_df.drop(axis=1, columns=['PatientID', 'TumourSite', 'TwoYearHNCDeath'])
y_opc_surv = completed_opc_df['TwoYearHNCDeath']
X_opc_surv_train_time, X_opc_surv_test_time, y_opc_surv_train, y_opc_surv_test = train_test_split(X_opc_surv, y_opc_surv, stratify=y_opc_surv, random_state=0, test_size = 0.2)
# remove survival time from X train and test
X_opc_surv_train = X_opc_surv_train_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_opc_surv_train)
X_opc_surv_test = X_opc_surv_test_time.drop(axis=1, columns=['SurvivalMonths'])
# print(X_opc_surv_test)

# use gridsearchCV to find for parameter tuning 
max_features = ["sqrt", "log2"]
n_estimators = [100, 1000, 10000, 100000]
class_weight = [{0:.1, 1:.9}, {0:.2, 1:0.8}, {0:.3, 1:.7}, {0:.4, 1:.6}, {0:.5, 1:.5}]

# create parameter dictionary for tuning
opc_surv_rf_param_grid = dict(max_features=max_features,
                            n_estimators=n_estimators, 
                            class_weight=class_weight)

# define the gridsearchCV object with 5 CV
opc_surv_grid_search_rf = GridSearchCV(rf, param_grid=opc_surv_rf_param_grid, cv=5, n_jobs=-1, scoring = 'accuracy')

In [ ]:
# fit the tree gridsearchCV object
opc_surv_grid_search_rf.fit(X_opc_surv_train, y_opc_surv_train)

In [ ]:
# print best parameters and average accuracy score
print("Best parameters: {}".format(opc_surv_grid_search_rf.best_params_))
print("Best cross-validation average accuracy score: {:.2f}".format(opc_surv_grid_search_rf.best_score_))

# predict outcome using tuned model on train set
y_pred_opc_surv_rf_grid_search_train = opc_surv_grid_search_rf.predict(X_opc_surv_train)
# predict probabilities on train set
y_prob_opc_surv_rf_grid_search_train = opc_surv_grid_search_rf.predict_proba(X_opc_surv_train)
# print(y_prob_opc_surv_rf_grid_search_train)

# calculate percentiles on train set
opc_surv_train_25_percentile = np.percentile (y_prob_opc_surv_rf_grid_search_train[:,1], 25) # for prediction of class 1 'death'
# print('25th percentile:', opc_surv_train_25_percentile)
opc_surv_train_50_percentile = np.percentile (y_prob_opc_surv_rf_grid_search_train[:,1], 50)
# print('50th percentile:', opc_surv_train_50_percentile)
opc_surv_train_75_percentile = np.percentile (y_prob_opc_surv_rf_grid_search_train[:,1], 75)
# print('75th percentile:', opc_surv_train_75_percentile)

# predict outcome using tuned model on test set
y_pred_opc_surv_rf_grid_search_test = opc_surv_grid_search_rf.predict(X_opc_surv_test)
# print(y_pred_opc_surv_rf_grid_search_test)
# predict probabilities on test set
y_prob_opc_surv_rf_grid_search_test = opc_surv_grid_search_rf.predict_proba(X_opc_surv_test)
# print(y_prob_opc_surv_rf_grid_search_test[:,1])

# calculate percentiles on test set
X_opc_surv_test_25_percentile = np.percentile (y_prob_opc_surv_rf_grid_search_test[:,1], 25)
# print('25th percentile:', X_opc_surv_test_25_percentile)
X_opc_surv_test_50_percentile = np.percentile (y_prob_opc_surv_rf_grid_search_test[:,1], 50)
# print('50th percentile:', X_opc_surv_test_50_percentile)
X_opc_surv_test_75_percentile = np.percentile (y_prob_opc_surv_rf_grid_search_test[:,1], 75)
# print('75th percentile:', X_opc_surv_test_75_percentile)

# print the accuracy scores
print("Accuracy on training set: {:.2f}".format(accuracy_score(y_opc_surv_train, y_pred_opc_surv_rf_grid_search_train)))
print("Accuracy on test set: {:.2f}".format(accuracy_score(y_opc_surv_test, y_pred_opc_surv_rf_grid_search_test)))

In [ ]:
# evaluate performance metrics of the tuned model, construct the confusion matrix
cm_opc_surv_rf = confusion_matrix(y_true = y_opc_surv_test, y_pred = y_pred_opc_surv_rf_grid_search_test)
# plot confusion matrix
print("Confusion matrix plot of the rf:")
print(cm_opc_surv_rf)
print()
print("Confusion matrix plot of the rf:")
disease_labels = ['HNC death Absent', 'HNC death Present']
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(cm_opc_surv_rf)
plt.title('Confusion matrix of the rf:')
fig.colorbar(cax)
ax.set_xticklabels([''] + disease_labels)
ax.set_yticklabels([''] + disease_labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
# print classification report
print("Precision, Recall, F1-score for class 0 (HNC death Absent) and class 1 (HNC death Present) classes of the test set:")
print("Using random forest model")
print()
print(classification_report(y_opc_surv_test, y_pred_opc_surv_rf_grid_search_test))

In [ ]:
# create auc roc curve
y_pred_proba_opc_surv_rf = opc_surv_grid_search_rf.predict_proba(X_opc_surv_test)[:,1]
print("Area under curve for random forest:")
print(metrics.roc_auc_score(y_true = y_opc_surv_test, y_score = y_pred_opc_surv_rf_grid_search_test))
fpr, tpr, thresholds = metrics.roc_curve(y_opc_surv_test, y_pred_opc_surv_rf_grid_search_test)
plt.figure()
plt.plot(fpr, tpr, color='red', lw=2, label='ROC curve random forest')
plt.plot([0, 1], [0, 1], color='blue', lw=2, linestyle='--')
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.title('ROC curve')
plt.show()

In [ ]:
# save the model to disk
pickle.dump(opc_surv_grid_search_rf, open('/PATH/TO/FILE.pkl', 'wb'))

In [ ]:
# convert pred/proba arrays to dataframes and add column labels
X_opc_surv_test_pred_df = pd.DataFrame([y_pred_opc_surv_rf_grid_search_test]).transpose()
X_opc_surv_test_pred_df.columns = ['PredictedOutcome']
X_opc_surv_test_proba_df = pd.DataFrame([y_prob_opc_surv_rf_grid_search_test[:,1]]).transpose()
X_opc_surv_test_proba_df.columns = ['PredictedProbability']

# append predicted outcome and probability to test set using indexes
opc_km_df = X_opc_surv_test.copy()
opc_km_df['SurvivalMonths'] = X_opc_surv_test_time['SurvivalMonths']
opc_km_df['TwoYearHNCDeath']= y_opc_surv_test
opc_km_df.reset_index(inplace=True)
opc_km_df['PredictedOutcome'] = X_opc_surv_test_pred_df
opc_km_df['PredictedProbability'] = X_opc_surv_test_proba_df
# print(opc_km_df)

# create risk group variable, 3=poor prognosis, 2=medium prognosis, 1=good prognosis
opc_km_df['RiskGroup'] = [3 if x<opc_surv_train_25_percentile else 1 if x>opc_surv_train_75_percentile else 2 for x in opc_km_df['PredictedProbability']]
# print(opc_km_df)

In [ ]:
# plot km curve for risk group
fig, ax = plt.subplots(figsize=(6,6), dpi=80)

df = opc_km_df
ix = df['RiskGroup'] == 'riskgroup'

opc_km_rg1 = KaplanMeierFitter()
ax = opc_km_rg1.fit(df[df["RiskGroup"] == 1]['SurvivalMonths'], df[df["RiskGroup"] == 1]['TwoYearHNCDeath'], label='poor prognosis').plot_survival_function(ax=ax)

opc_km_rg2 = KaplanMeierFitter()
ax = opc_km_rg2.fit(df[df["RiskGroup"] == 2]['SurvivalMonths'], df[df["RiskGroup"] == 2]['TwoYearHNCDeath'], label='medium prognosis').plot_survival_function(ax=ax)

opc_km_rg3 = KaplanMeierFitter()
ax = opc_km_rg3.fit(df[df["RiskGroup"] == 3]['SurvivalMonths'], df[df["RiskGroup"] == 3]['TwoYearHNCDeath'], label='good prognosis').plot_survival_function(ax=ax)

from lifelines.plotting import add_at_risk_counts
add_at_risk_counts(opc_km_rg1, opc_km_rg2, opc_km_rg3, ax=ax)
ax.set_xlabel("Survival time (months)")
ax.set_xlim(0, 24) # set x axis to 24 months
ax.set_ylabel("Survival")
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='lower left')
plt.tight_layout()

# save km figure
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
# print event table for each risk group
print(opc_km_rg1.event_table)
print(opc_km_rg2.event_table)
print(opc_km_rg3.event_table)

In [ ]:
# calculate log rank statistics
opc_results = multivariate_logrank_test(opc_km_df['SurvivalMonths'], opc_km_df['RiskGroup'], opc_km_df['TwoYearHNCDeath'])
opc_results.print_summary()
print("p value =", opc_results.p_value)        
print("t statistic =", opc_results.test_statistic) 

In [ ]:
# calculate c-index
opc_surv_cindex = concordance_index(opc_km_df['SurvivalMonths'], -opc_km_df['PredictedProbability'], opc_km_df['TwoYearHNCDeath'])
print(opc_surv_cindex)

# calibration plot 

In [ ]:
# initiate models
# calibrated_hnc_model = CalibratedClassifierCV(hnc_grid_search_rf, method='sigmoid', cv=5)
# calibrated_hpc_model = CalibratedClassifierCV(hpc_grid_search_rf, method='sigmoid', cv=5)
# calibrated_lyx_model = CalibratedClassifierCV(lyx_grid_search_rf, method='sigmoid', cv=5)
# calibrated_npc_model = CalibratedClassifierCV(npc_grid_search_rf, method='sigmoid', cv=5)
# calibrated_occ_model = CalibratedClassifierCV(occ_grid_search_rf, method='sigmoid', cv=5)
# calibrated_opc_model = CalibratedClassifierCV(opc_grid_search_rf, method='sigmoid', cv=5)

# fit calibrated models
# calibrated_hnc_model.fit(X_hnc_train, y_hnc_train)
# calibrated_hpc_model.fit(X_hpc_train, y_hpc_train)
# calibrated_lyx_model.fit(X_lyx_train, y_lyx_train)
# calibrated_npc_model.fit(X_npc_train, y_npc_train)
# calibrated_occ_model.fit(X_occ_train, y_occ_train)
# calibrated_opc_model.fit(X_opc_train, y_opc_train)

# predict probabilities
hnc_probs = hnc_grid_search_rf.predict_proba(X_hnc_test)[:, 1]
hpc_probs = hpc_grid_search_rf.predict_proba(X_hpc_test)[:, 1]
lyx_probs = lyx_grid_search_rf.predict_proba(X_lyx_test)[:, 1]
npc_probs = npc_grid_search_rf.predict_proba(X_npc_test)[:, 1]
occ_probs = occ_grid_search_rf.predict_proba(X_occ_test)[:, 1]
opc_probs = opc_grid_search_rf.predict_proba(X_opc_test)[:, 1]

# reliability diagram
fop_hnc, mpv_hnc = calibration_curve(y_hnc_test, hnc_probs, n_bins=10, normalize=True)
fop_hpc, mpv_hpc = calibration_curve(y_hpc_test, hpc_probs, n_bins=10, normalize=True)
fop_lyx, mpv_lyx = calibration_curve(y_lyx_test, lyx_probs, n_bins=10, normalize=True)
fop_npc, mpv_npc = calibration_curve(y_npc_test, npc_probs, n_bins=10, normalize=True)
fop_occ, mpv_occ = calibration_curve(y_occ_test, occ_probs, n_bins=10, normalize=True)
fop_opc, mpv_opc = calibration_curve(y_opc_test, opc_probs, n_bins=10, normalize=True)

# plot perfectly calibrated
fig, ax = plt.subplots(figsize=(8,6), dpi=80)
plt.plot([0, 1], [0, 1], linestyle='--', label='perfectly calibrated')

# plot calibrated reliability
plt.plot(mpv_hnc, fop_hnc, marker='.', label='head and neck')
plt.plot(mpv_hpc, fop_hpc, marker='.', label='hypopharynx')
plt.plot(mpv_lyx, fop_lyx, marker='.', label='larynx')
plt.plot(mpv_npc, fop_npc, marker='.', label='nasopharynx')
plt.plot(mpv_occ, fop_occ, marker='.', label='oralcavity')
plt.plot(mpv_opc, fop_opc, marker='.', label='oropharynx')

ax.set_xlabel('Predicted probability')
ax.set_ylabel('Observed probability')
plt.legend()
plt.tight_layout()

# save figure
fig.savefig("/FILE/TO/PATH.jpeg", dpi=1200)
plt.show()

In [ ]:
# print probabilities
hnc_prob_y = y_hnc_test, y_pred_proba_hnc_rf
print('hnc prob:', hnc_prob_y)
hpc_prob_y = y_hpc_test, y_pred_proba_hpc_rf
print('hpc prob:', hpc_prob_y)
lyx_prob_y = y_lyx_test, y_pred_proba_lyx_rf
print('lyx prob:', lyx_prob_y)
npc_prob_y = y_npc_test, y_pred_proba_npc_rf
print('npc prob:', npc_prob_y)
occ_prob_y = y_occ_test, y_pred_proba_occ_rf
print('occ prob:', occ_prob_y)
opc_prob_y = y_opc_test, y_pred_proba_opc_rf
print('opc prob:', opc_prob_y)

In [ ]:
# calculate Brier score
# brier_score_loss(y_true, y_prob)
hnc_brier = brier_score_loss(y_hnc_test, y_pred_proba_hnc_rf)
print('hnc brier:', hnc_brier)
hpc_brier = brier_score_loss(y_hpc_test, y_pred_proba_hpc_rf)
print('hpc brier:', hpc_brier)
lyx_brier = brier_score_loss(y_lyx_test, y_pred_proba_lyx_rf)
print('lyx brier:', lyx_brier)
npc_brier = brier_score_loss(y_npc_test, y_pred_proba_npc_rf)
print('npc brier:', npc_brier)
occ_brier = brier_score_loss(y_occ_test, y_pred_proba_occ_rf)
print('occ brier:', occ_brier)
opc_brier = brier_score_loss(y_opc_test, y_pred_proba_opc_rf)
print('opc brier:', opc_brier)

In [ ]:
# run 1000 sample bootstrap to calculate 95% CI for Brier score
hnc_brier_score = []
n_iterations = 1000
for i in range(n_iterations):
    hnc_bs_y_true, hnc_bs_y_pred = resample(y_hnc_test, y_pred_proba_hnc_rf, replace=True)
    hnc_score = brier_score_loss(hnc_bs_y_true, hnc_bs_y_pred)
    hnc_brier_score.append(hnc_score)

# sns.kdeplot(hnc_brier_score)
# plt.show()

# calculate 95% CI of Brier score
hnc_brier_25_percentile = np.percentile(hnc_brier_score, 25)
print('25th percentile hnc:', hnc_brier_25_percentile)
hnc_brier_50_percentile = np.percentile(hnc_brier_score, 50) # value should be very close to calculated Brier score
print('50th percentile hnc:', hnc_brier_50_percentile)
hnc_brier_75_percentile = np.percentile(hnc_brier_score, 75)
print('75th percentile hnc:', hnc_brier_75_percentile)

hpc_brier_score = []
n_iterations = 1000
for i in range(n_iterations):
    hpc_bs_y_true, hpc_bs_y_pred = resample(y_hpc_test, y_pred_proba_hpc_rf, replace=True)
    hpc_score = brier_score_loss(hpc_bs_y_true, hpc_bs_y_pred)
    hpc_brier_score.append(hpc_score)

# sns.kdeplot(hpc_brier_score)
# plt.show()

# calculate 95% CI of Brier score
hpc_brier_25_percentile = np.percentile(hpc_brier_score, 25)
print('25th percentile hpc:', hpc_brier_25_percentile)
hpc_brier_50_percentile = np.percentile(hpc_brier_score, 50) # value should be very close to calculated Brier score
print('50th percentile hpc:', hpc_brier_50_percentile)
hpc_brier_75_percentile = np.percentile(hpc_brier_score, 75)
print('75th percentile hpc:', hpc_brier_75_percentile)

lyx_brier_score = []
n_iterations = 1000
for i in range(n_iterations):
    lyx_bs_y_true, lyx_bs_y_pred = resample(y_lyx_test, y_pred_proba_lyx_rf, replace=True)
    lyx_score = brier_score_loss(lyx_bs_y_true, lyx_bs_y_pred)
    lyx_brier_score.append(lyx_score)

# sns.kdeplot(lyx_brier_score)
# plt.show()

# calculate 95% CI of Brier score
lyx_brier_25_percentile = np.percentile(lyx_brier_score, 25)
print('25th percentile lyx:', lyx_brier_25_percentile)
lyx_brier_50_percentile = np.percentile(lyx_brier_score, 50) # value should be very close to calculated Brier score
print('50th percentile lyx:', lyx_brier_50_percentile)
lyx_brier_75_percentile = np.percentile(lyx_brier_score, 75)
print('75th percentile lyx:', lyx_brier_75_percentile)

npc_brier_score = []
n_iterations = 1000
for i in range(n_iterations):
    npc_bs_y_true, npc_bs_y_pred = resample(y_npc_test, y_pred_proba_npc_rf, replace=True)
    npc_score = brier_score_loss(npc_bs_y_true, npc_bs_y_pred)
    npc_brier_score.append(npc_score)

# sns.kdeplot(npc_brier_score)
# plt.show()

# calculate 95% CI of Brier score
npc_brier_25_percentile = np.percentile(npc_brier_score, 25)
print('25th percentile npc:', npc_brier_25_percentile)
npc_brier_50_percentile = np.percentile(npc_brier_score, 50) # value should be very close to calculated Brier score
print('50th percentile npc:', npc_brier_50_percentile)
npc_brier_75_percentile = np.percentile(npc_brier_score, 75)
print('75th percentile npc:', npc_brier_75_percentile)

occ_brier_score = []
n_iterations = 1000
for i in range(n_iterations):
    occ_bs_y_true, occ_bs_y_pred = resample(y_occ_test, y_pred_proba_occ_rf, replace=True)
    occ_score = brier_score_loss(occ_bs_y_true, occ_bs_y_pred)
    occ_brier_score.append(occ_score)

# sns.kdeplot(occ_brier_score)
# plt.show()

# calculate 95% CI of Brier score
occ_brier_25_percentile = np.percentile(occ_brier_score, 25)
print('25th percentile occ:', occ_brier_25_percentile)
occ_brier_50_percentile = np.percentile(occ_brier_score, 50) # value should be very close to calculated Brier score
print('50th percentile occ:', occ_brier_50_percentile)
occ_brier_75_percentile = np.percentile(occ_brier_score, 75)
print('75th percentile occ:', occ_brier_75_percentile)

opc_brier_score = []
n_iterations = 1000
for i in range(n_iterations):
    opc_bs_y_true, opc_bs_y_pred = resample(y_opc_test, y_pred_proba_opc_rf, replace=True)
    opc_score = brier_score_loss(opc_bs_y_true, opc_bs_y_pred)
    opc_brier_score.append(opc_score)

# sns.kdeplot(opc_brier_score)
# plt.show()

# calculate 95% CI of Brier score
opc_brier_25_percentile = np.percentile(opc_brier_score, 25)
print('25th percentile opc:', opc_brier_25_percentile)
opc_brier_50_percentile = np.percentile(opc_brier_score, 50) # value should be very close to calculated Brier score
print('50th percentile opc:', opc_brier_50_percentile)
opc_brier_75_percentile = np.percentile(opc_brier_score, 75)
print('75th percentile opc:', opc_brier_75_percentile)

# feature importances

In [ ]:
# print feature importance array
print("Feature importances:\n{}".format(hnc_grid_search_rf.best_estimator_.feature_importances_))
print("Feature importances:\n{}".format(hpc_grid_search_rf.best_estimator_.feature_importances_))
print("Feature importances:\n{}".format(lyx_grid_search_rf.best_estimator_.feature_importances_))
print("Feature importances:\n{}".format(npc_grid_search_rf.best_estimator_.feature_importances_))
print("Feature importances:\n{}".format(occ_grid_search_rf.best_estimator_.feature_importances_))
print("Feature importances:\n{}".format(opc_grid_search_rf.best_estimator_.feature_importances_))

In [ ]:
# convert array to dataframe and sort feature importance
hnc_df_feature_importance = pd.DataFrame(hnc_grid_search_rf.best_estimator_.feature_importances_, index=X_hnc_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)
hpc_df_feature_importance = pd.DataFrame(hpc_grid_search_rf.best_estimator_.feature_importances_, index=X_hpc_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)
lyx_df_feature_importance = pd.DataFrame(lyx_grid_search_rf.best_estimator_.feature_importances_, index=X_lyx_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)
npc_df_feature_importance = pd.DataFrame(npc_grid_search_rf.best_estimator_.feature_importances_, index=X_npc_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)
occ_df_feature_importance = pd.DataFrame(occ_grid_search_rf.best_estimator_.feature_importances_, index=X_occ_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)
opc_df_feature_importance = pd.DataFrame(opc_grid_search_rf.best_estimator_.feature_importances_, index=X_opc_smote.columns, columns=['feature importance']).sort_values('feature importance', ascending=True)

In [ ]:
# print sorted feature importance
print("hnc feature importance:", hnc_df_feature_importance)
print("hpc feature importance:", hpc_df_feature_importance)
print("lyx feature importance:", lyx_df_feature_importance)
print("npc feature importance:", npc_df_feature_importance)
print("occ feature importance:", occ_df_feature_importance)
print("opc feature importance:", opc_df_feature_importance)

In [ ]:
# calculate feature importance
hnc = hnc_grid_search_rf.best_estimator_.feature_importances_
hpc = hpc_grid_search_rf.best_estimator_.feature_importances_
lyx = lyx_grid_search_rf.best_estimator_.feature_importances_
npc = npc_grid_search_rf.best_estimator_.feature_importances_
occ = occ_grid_search_rf.best_estimator_.feature_importances_
opc = opc_grid_search_rf.best_estimator_.feature_importances_

In [ ]:
# select columns to plot for hnc site
index_hnc = ['EQD2T', 'N classification', 'T classification', 'Tumour site', 'Hospital location', 'Gender','Age']
hnc_fi_df = pd.DataFrame({'head and neck': hnc
                         }, 
                   index=index_hnc)
# plot feature importances
ax = hnc_fi_df.plot.barh(width=0.9)
# save figure
plt.tight_layout()
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()

In [ ]:
index_subsite = ['EQD2T', 'N classification', 'T classification', 'Hospital location', 'Gender','Age']
subsite_fi_df = pd.DataFrame({'oropharynx': opc,
                              'oral cavity': occ,
                              'nasopharynx': npc,
                              'larynx': lyx,                         
                              'hypopharynx': hpc
                             }, 
                   index=index_subsite)

ax = subsite_fi_df.plot(kind='barh', width=0.9)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], fontsize=8.5, loc='best', bbox_to_anchor=(0.5, 0.4, 0.5, 0.5))
# save figure
plt.tight_layout()
plt.savefig("/PATH/TO/FILE.jpeg", dpi=1200)
plt.show()